<a href="https://colab.research.google.com/github/aisyahaini/MachineLearning/blob/main/Text_Extraction_Aisyah_Nuraini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text



In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from collections import Counter
import scipy.sparse as sp
from numpy.linalg import norm
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Corpus

In [ ]:
# corpus = [
# "Love is like pi - natural, irrational, and very important.",
# "Love is being stupid together.",
# "Love is sharing your popcorn.",
# "Love is like Heaven, but it can hurt like Hell."
# ]

# corpus = [
#     "Love is like pi - natural, irrational, and very important."
# ]

corpus = [
    "love song ï»¿",
    "lets make first female reach one billion!! Share and replay! ï»¿",
    "shakira best world cupï»¿",
    "best world cup song ever!!!!ï»¿",
    "SEE SONG OPEN GOOGLE AND TYPE Shakira GuruOfMovieï»¿",
    "Awesome ï»¿",
    "like shakira..ï»¿",
    "Shakira - Waka Waka LOVE SONG!!!!!!!!!!!!!!!ï»¿",
    "Why many disliked??????!!!!!!ðŸ˜¯ï»¿",
    "totally love song. absolutely fantastic! love ya shakira!ï»¿"
]


## Preprocessing

### Cleaning

In [ ]:
cleaned_corpus = [re.sub(r'[^a-zA-Z0-9\s]', '', sentence) for sentence in corpus] # Menghapus semua karakter non-alphanumeric dari setiap string dalam corpus.
cleaned_corpus = [sentence.lower().strip() for sentence in cleaned_corpus] # Menghilangkan Whitespace dan menjadikan teks lowercase.

for i in cleaned_corpus :
    print(i)

love song
lets make first female reach one billion share and replay
shakira best world cup
best world cup song ever
see song open google and type shakira guruofmovie
awesome
like shakira
shakira  waka waka love song
why many disliked
totally love song absolutely fantastic love ya shakira


### Stemming

In [ ]:
stemmer = PorterStemmer()

import nltk
nltk.download('punkt')

stemmed_corpus = []
for sentence in corpus:
    words = word_tokenize(sentence)
    stemmed_words = [stemmer.stem(word) for word in words]
    stemmed_sentence = ' '.join(stemmed_words)
    stemmed_corpus.append(stemmed_sentence)

for i, stemmed_sentence in enumerate(stemmed_corpus):
    print(f"Sentence {i+1}: {stemmed_sentence}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Sentence 1: love song ï » ¿
Sentence 2: let make first femal reach one billion ! ! share and replay ! ï » ¿
Sentence 3: shakira best world cupï » ¿
Sentence 4: best world cup song ever ! ! ! ! ï » ¿
Sentence 5: see song open googl and type shakira guruofmovieï » ¿
Sentence 6: awesom ï » ¿
Sentence 7: like shakira .. ï » ¿
Sentence 8: shakira - waka waka love song ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ï » ¿
Sentence 9: whi mani dislik ? ? ? ? ? ? ! ! ! ! ! ! ðÿ˜¯ï » ¿
Sentence 10: total love song . absolut fantast ! love ya shakira ! ï » ¿


### Tokenization

In [ ]:
tokens = [nltk.word_tokenize(sentence) for sentence in cleaned_corpus]
for i in tokens :
    print(i)

['love', 'song']
['lets', 'make', 'first', 'female', 'reach', 'one', 'billion', 'share', 'and', 'replay']
['shakira', 'best', 'world', 'cup']
['best', 'world', 'cup', 'song', 'ever']
['see', 'song', 'open', 'google', 'and', 'type', 'shakira', 'guruofmovie']
['awesome']
['like', 'shakira']
['shakira', 'waka', 'waka', 'love', 'song']
['why', 'many', 'disliked']
['totally', 'love', 'song', 'absolutely', 'fantastic', 'love', 'ya', 'shakira']


### Stopwords Removal

In [ ]:
stop_words = nltk.corpus.stopwords.words('english')
filtered_tokens = [[word for word in sentence if word.lower() not in stop_words] for sentence in tokens]

filtered_text = [' '.join(sentence) for sentence in filtered_tokens]

for i in filtered_text :
    print(i)

love song
lets make first female reach one billion share replay
shakira best world cup
best world cup song ever
see song open google type shakira guruofmovie
awesome
like shakira
shakira waka waka love song
many disliked
totally love song absolutely fantastic love ya shakira


## Feature Extraction

### TF-IDF

In [ ]:
class TFIDF(object):

    def __init__(self, corpus):
        self.corpus = corpus
        self.norm_corpus  = None

    def __normalize_corpus(self, d):
        stop_words = nltk.corpus.stopwords.words('english')
        d = re.sub(r'[^a-zA-Z0-9\s]', '', d, re.I|re.A)
        d = d.lower().strip()
        tks = nltk.word_tokenize(d)
        f_tks = [t for t in tks if t not in stop_words]
        return ' '.join(f_tks)

    def preprocessing_text(self):
        n_c = np.vectorize(self.__normalize_corpus)
        self.norm_corpus = n_c(self.corpus)

    def tf(self):
        words_array = [doc.split() for doc in self.norm_corpus]
        words = list(set([word for words in words_array for word in words]))
        features_dict = {w:0 for w in words}
        tf = []
        for doc in self.norm_corpus:
            bowf_doc = Counter(doc.split())
            all_f = Counter(features_dict)
            bowf_doc.update(all_f)
            tf.append(bowf_doc)
        return pd.DataFrame(tf)

    def df(self, tf):
        features_names = list(tf.columns)
        df = np.diff(sp.csc_matrix(tf, copy=True).indptr)
        df = 1 + df
        return df

    def idf(self, df):
        N = 1 + len(self.norm_corpus)
        idf = (1.0 + np.log(float(N) / df))
        idf_d = sp.spdiags(idf, diags= 0, m=len(df), n= len(df)).todense()
        return idf, idf_d

    def tfidf(self, tf, idf):
        tf = np.array(tf, dtype='float64')
        print("TF : ", tf)
        print("IDF : ", idf)
        tfidf = tf * idf
        print("TFIDF : ", tfidf)
        norms = norm(tfidf , axis=1)
        print("Normal : ", norms)
        return (tfidf / norms[:,None])

In [ ]:
obj = TFIDF(corpus)
obj.preprocessing_text()

#### TF (Term Frequency)

In [ ]:
tf = obj.tf()
tf.head(10000)

,love,song,shakira,first,ever,like,lets,fantastic,google,ya,...,make,best,awesome,many,world,billion,absolutely,totally,share,replay
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,1,0,0,0,...,1,0,0,0,0,1,0,0,1,1
2,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
3,0,1,0,0,1,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
4,0,1,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
6,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
9,2,1,1,0,0,0,0,1,0,1,...,0,0,0,0,0,0,1,1,0,0


In [ ]:
df = obj.df(tf)

temp_columns = tf.columns.values.reshape(-1,1)
combine = np.hstack((temp_columns, df.reshape(-1, 1)))
temp_df = pd.DataFrame(combine, columns = ["", ""])
print(temp_df)

                  
0          love  4
1          song  6
2       shakira  6
3         first  2
4          ever  2
5          like  2
6          lets  2
7     fantastic  2
8        google  2
9            ya  2
10       female  2
11         open  2
12          cup  3
13        reach  2
14          one  2
15         type  2
16          see  2
17         waka  2
18  guruofmovie  2
19     disliked  2
20         make  2
21         best  3
22      awesome  2
23         many  2
24        world  3
25      billion  2
26   absolutely  2
27      totally  2
28        share  2
29       replay  2


#### IDF (Inverse Document Frequency)

In [ ]:
idf, idf_d = obj.idf(df)

combine = np.hstack((temp_columns, idf.reshape(-1, 1)))
temp_df = pd.DataFrame(combine, columns = ["", ""])
print(temp_df)

                         
0          love  2.011601
1          song  1.606136
2       shakira  1.606136
3         first  2.704748
4          ever  2.704748
5          like  2.704748
6          lets  2.704748
7     fantastic  2.704748
8        google  2.704748
9            ya  2.704748
10       female  2.704748
11         open  2.704748
12          cup  2.299283
13        reach  2.704748
14          one  2.704748
15         type  2.704748
16          see  2.704748
17         waka  2.704748
18  guruofmovie  2.704748
19     disliked  2.704748
20         make  2.704748
21         best  2.299283
22      awesome  2.704748
23         many  2.704748
24        world  2.299283
25      billion  2.704748
26   absolutely  2.704748
27      totally  2.704748
28        share  2.704748
29       replay  2.704748


#### TF-IDF

In [ ]:
tfidf = obj.tfidf(tf, idf)

TF :  [[1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 1. 0. 0. 1. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1.
  0. 0. 0. 0. 0. 0.]
 [2. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 1. 1. 0. 0.]]
IDF :  [2.01160091 1.6061358  1.6061358  2.

In [ ]:
df = pd.DataFrame(np.round(tfidf,2), columns= list(tf.columns))
df.head(1000)

,love,song,shakira,first,ever,like,lets,fantastic,google,ya,...,make,best,awesome,many,world,billion,absolutely,totally,share,replay
0,0.78,0.62,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.00,0.33,0.00,0.00,0.33,0.00,0.00,0.00,...,0.33,0.00,0.0,0.00,0.00,0.33,0.00,0.00,0.33,0.33
2,0.00,0.00,0.37,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.54,0.0,0.00,0.54,0.00,0.00,0.00,0.00,0.00
3,0.00,0.32,0.00,0.00,0.53,0.00,0.00,0.00,0.00,0.00,...,0.00,0.45,0.0,0.00,0.45,0.00,0.00,0.00,0.00,0.00
4,0.00,0.25,0.25,0.00,0.00,0.00,0.00,0.00,0.42,0.00,...,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,1.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,0.00,0.00,0.51,0.00,0.00,0.86,0.00,0.00,0.00,0.00,...,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,0.32,0.26,0.26,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.0,0.71,0.00,0.00,0.00,0.00,0.00,0.00
9,0.57,0.23,0.23,0.00,0.00,0.00,0.00,0.38,0.00,0.38,...,0.00,0.00,0.0,0.00,0.00,0.00,0.38,0.38,0.00,0.00


## Decomposition

In [ ]:
from sklearn.decomposition import PCA

df_new = df.copy()

n_components = 10

pca = PCA(n_components=n_components)

# Terapkan PCA pada DataFrame Anda
pca_result = pca.fit_transform(df_new)

# Ubah hasil PCA menjadi DataFrame
pca_df = pd.DataFrame(data=pca_result, columns=[f'PC{i+1}' for i in range(n_components)])

# Tampilkan hasil PCA
pca_df.head(1000)

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10
0,-0.553682,-0.361448,-0.227455,-0.005063,0.007403,0.044484,-0.112031,0.406781,-0.170142,3.805020e-17
1,0.061974,0.472646,-0.189118,-0.118373,0.824961,-0.021128,0.001196,-0.006816,-0.005044,3.805020e-17
2,0.686915,-0.387587,0.012041,0.003166,-0.006634,-0.079083,-0.002315,-0.200264,-0.276466,3.805020e-17
3,0.591385,-0.468456,-0.226145,-0.004979,0.007288,0.070397,-0.002986,0.178693,0.276606,3.805020e-17
4,-0.068983,0.073850,0.464867,0.014604,-0.023552,0.781099,-0.021285,-0.044007,0.005023,3.805020e-17
5,0.064084,0.496303,-0.252071,-0.661369,-0.471462,-0.017188,0.001113,-0.006583,-0.004920,3.805020e-17
6,0.075381,0.177473,0.737116,0.025133,-0.042833,-0.477924,-0.136682,0.170475,0.037592,3.805020e-17
7,-0.394697,-0.210152,0.064875,0.004349,-0.008424,-0.146682,0.703542,-0.127067,0.042085,3.805020e-17
8,0.064995,0.506755,-0.292142,0.742962,-0.286159,-0.015962,0.001082,-0.006492,-0.004870,3.805020e-17
9,-0.527371,-0.299383,-0.091968,-0.000429,-0.000589,-0.138014,-0.431635,-0.364719,0.100136,3.805020e-17


## Selection

In [ ]:
df_new.mean().sort_values(ascending=False)

song           0.168
love           0.167
shakira        0.162
awesome        0.100
world          0.099
best           0.099
cup            0.099
waka           0.087
like           0.086
many           0.071
disliked       0.071
ever           0.053
guruofmovie    0.042
see            0.042
type           0.042
open           0.042
google         0.042
ya             0.038
fantastic      0.038
absolutely     0.038
totally        0.038
one            0.033
reach          0.033
female         0.033
make           0.033
lets           0.033
first          0.033
billion        0.033
share          0.033
replay         0.033
dtype: float64

In [ ]:
import pandas as pd

# Misal df adalah DataFrame Anda dengan hasil TF-IDF
# Misal df adalah matriks 34x235

# Hitung rata-rata TF-IDF untuk setiap fitur
average_tfidf = df_new.mean()

# Inisialisasi set fitur yang dipilih
selected_features = set(df.columns)

# Lakukan backward selection
while len(selected_features) > 10:
    # Ambil subset DataFrame dengan fitur yang dipilih
    df_subset = df[list(selected_features)]

    # Hitung rata-rata TF-IDF untuk setiap fitur subset
    average_tfidf_subset = df_subset.mean()

    # Temukan fitur dengan rata-rata TF-IDF terendah
    feature_to_remove = average_tfidf_subset.idxmin()

    # Hapus fitur dengan rata-rata TF-IDF terendah dari set fitur terpilih
    selected_features.remove(feature_to_remove)

# Cetak fitur terpilih pada akhir backward selection
print("Fitur terpilih akhir:", selected_features)

Fitur terpilih akhir: {'shakira', 'like', 'cup', 'love', 'song', 'waka', 'best', 'awesome', 'many', 'world'}


In [ ]:
import pandas as pd

# Mengonversi set selected_features menjadi list
selected_features_list = list(selected_features)

# Membuat DataFrame baru dengan fitur-fitur terpilih
df_selected = df[selected_features_list]

# Cetak DataFrame baru
print("DataFrame baru dengan fitur terpilih:")
print(df_selected)


DataFrame baru dengan fitur terpilih:
   shakira  like   cup  love  song  waka  best  awesome  many  world
0     0.00  0.00  0.00  0.78  0.62  0.00  0.00      0.0  0.00   0.00
1     0.00  0.00  0.00  0.00  0.00  0.00  0.00      0.0  0.00   0.00
2     0.37  0.00  0.54  0.00  0.00  0.00  0.54      0.0  0.00   0.54
3     0.00  0.00  0.45  0.00  0.32  0.00  0.45      0.0  0.00   0.45
4     0.25  0.00  0.00  0.00  0.25  0.00  0.00      0.0  0.00   0.00
5     0.00  0.00  0.00  0.00  0.00  0.00  0.00      1.0  0.00   0.00
6     0.51  0.86  0.00  0.00  0.00  0.00  0.00      0.0  0.00   0.00
7     0.26  0.00  0.00  0.32  0.26  0.87  0.00      0.0  0.00   0.00
8     0.00  0.00  0.00  0.00  0.00  0.00  0.00      0.0  0.71   0.00
9     0.23  0.00  0.00  0.57  0.23  0.00  0.00      0.0  0.00   0.00
